## Agent Chat Lab -  Multi Agent Orchestration

In [1]:
from azure.ai.projects.models import AzureAISearchTool, BingGroundingTool
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
import sys
import os
from dotenv import load_dotenv

sys.path.insert(1, "..")
from src.agent import Agent
from src.multiagent import MultiAgent

load_dotenv(override=True)

True

In [2]:
# // Connect to Azure Foundry and extract the AI Search connection id
connection_string = os.environ["PROJECT_CONNECTION_STRING"]

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(),
    conn_str=connection_string,
)

# Extract the connection list.
conn_list = project_client.connections._list_connections()["value"]
conn_id = ""
bing_conn_id = ""

# Search in the metadata field of each connection in the list for the azure_ai_search type and get the id value to establish the variable
for conn in conn_list:
    category = conn["properties"].get("category", None)
    if category == "CognitiveSearch":
        conn_id = conn["id"]
    if category == "ApiKey":
        bing_conn_id = conn["id"]

print(f" Search Connection ID: {conn_id}")
print(f" Bing Connection ID: {bing_conn_id}")

 Search Connection ID: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/rg-lab-agents/providers/Microsoft.MachineLearningServices/workspaces/project-demo-7zmh/connections/hub-demo-7zmh-connection-AISearch
 Bing Connection ID: /subscriptions/06d043e2-5a2e-46bf-bf48-fffee525f377/resourceGroups/rg-lab-agents/providers/Microsoft.MachineLearningServices/workspaces/project-demo-7zmh/connections/hub-demo-7zmh-connection-BingSearch


In [4]:
# // Create agents
AGENT_SCORE = "Score"
AGENT_DEBT = "Dividas"
AGENT_CC_SANTANDER = "CC_Santander"
AGENT_CC_WILLIAMS = "CC_Will"
AGENT_CC_PICPAY = "CC_PicPay"
AGENT_CC_NEON = "CC_Neon"
AGENT_CC_ITAU = "CC_Itau"
AGENT_CC_DM = "CC_DM"
AGENT_CC_DIGIO = "CC_Digio"
AGENT_BING = "BING"

# // Create the tools
score_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="score",
)

debt_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="debt",
)

will_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaowill",
)

santander_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaosantander",
)

picpay_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaopicpay",
)

neon_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaoneon",
)

itau_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaoitau",
)

dm_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaodm",
)

digio_search_tool = AzureAISearchTool(
    index_connection_id=conn_id,
    index_name="cartaodigio",
)

bing_search_tool = BingGroundingTool(connection_id=bing_conn_id)

# // Azure AI Foundry Agent Creator
model_deployment_name = os.environ.get("AZURE_OPENAI_4o_MODEL_NAME")

async with Agent(model_deployment_name, connection_string) as agent_creator:
    agent_1 = await agent_creator.create_agent(
        name=AGENT_SCORE,
        instructions="You answer questions about Score",
        tool={
            "definitions": score_search_tool.definitions,
            "resources": score_search_tool.resources,
        },
    )

    agent_2 = await agent_creator.create_agent(
        name=AGENT_DEBT,
        instructions="You answer questions about Debt using the Azure AI Search tool",
        tool={
            "definitions": debt_search_tool.definitions,
            "resources": debt_search_tool.resources,
        },
    )

    agent_3 = await agent_creator.create_agent(
        name=AGENT_CC_SANTANDER,
        instructions="You answer questions about Santander Credit Card",
        tool={
            "definitions": santander_search_tool.definitions,
            "resources": santander_search_tool.resources,
        },
    )

    agent_4 = await agent_creator.create_agent(
        name=AGENT_CC_WILLIAMS,
        instructions="You answer questions about Williams Credit Card",
        tool={
            "definitions": will_search_tool.definitions,
            "resources": will_search_tool.resources,
        },
    )

    agent_5 = await agent_creator.create_agent(
        name=AGENT_CC_PICPAY,
        instructions="You answer questions about PicPay Credit Card",
        tool={
            "definitions": picpay_search_tool.definitions,
            "resources": picpay_search_tool.resources,
        },
    )

    agent_6 = await agent_creator.create_agent(
        name=AGENT_CC_NEON,
        instructions="You answer questions about Neon Credit Card",
        tool={
            "definitions": neon_search_tool.definitions,
            "resources": neon_search_tool.resources,
        },
    )

    agent_7 = await agent_creator.create_agent(
        name=AGENT_CC_ITAU,
        instructions="You answer questions about Itau Credit Card",
        tool={
            "definitions": itau_search_tool.definitions,
            "resources": itau_search_tool.resources,
        },
    )

    agent_8 = await agent_creator.create_agent(
        name=AGENT_CC_DM,
        instructions="You answer questions about DM Credit Card",
        tool={
            "definitions": dm_search_tool.definitions,
            "resources": dm_search_tool.resources,
        },
    )

    agent_9 = await agent_creator.create_agent(
        name=AGENT_CC_DIGIO,
        instructions="You answer questions about Digio Credit Card",
        tool={
            "definitions": digio_search_tool.definitions,
            "resources": digio_search_tool.resources,
        },
    )

    agent_10 = await agent_creator.create_agent(
        name=AGENT_BING,
        instructions="You answer questions using Bing Search",
        tool={
            "definitions": bing_search_tool.definitions,
            "resources": bing_search_tool.resources,
        },
    )

In [5]:
# // Selection function
selection_template = f"""
    Determine which participant takes the next turn in a conversation based on the the most recent participant.
    State only the name of the participant to take the next turn.
    No participant should take more than one turn in a row.
    
    Choose only from these participants:
    - {AGENT_SCORE}
    - {AGENT_DEBT}
    - {AGENT_CC_SANTANDER}
    - {AGENT_CC_WILLIAMS}
    - {AGENT_CC_PICPAY}
    - {AGENT_CC_NEON}
    - {AGENT_CC_ITAU}
    - {AGENT_CC_DM}
    - {AGENT_CC_DIGIO}
    - {AGENT_BING}

    History:
    {{{{$history}}}}
"""

termination_template = f"""
    Determine if the question has been answered.  If so, respond with a single word: yes

    History:
    {{{{$history}}}}
"""

In [6]:
maximum_iterations = 2

multi_agent = MultiAgent(
    model_deployment_name, maximum_iterations, selection_template, termination_template
)
group = multi_agent.create_chat_group(
    agents=[
        agent_1,
        agent_2,
        agent_3,
        agent_4,
        agent_5,
        agent_6,
        agent_7,
        agent_8,
        agent_9,
        agent_10,
    ]
)

In [3]:
# // Delete the agents
agent_list = project_client.agents.list_agents().data
for agent in agent_list:
    project_client.agents.delete_agent(agent.id)